In [1]:
import torch

In [2]:
from model import Model

/Users/wangzongcheng/wzc/code/latent-v4/model.py:9: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [3]:
from config import load_config

In [6]:
config = load_config("./config.yaml")

In [7]:
model = Model(config)

In [8]:
model.params_count(config)

({'encoder_params': 12070912,
  'aux_decocder_params': 6039552,
  'main_params': 30164992,
  'decoder_params': 6047744,
  'parallel_params': 7604224,
  'ar_decoder_params': 9178112,
  'model_params': 71105536,
  'vocab_params': 64685568,
  'total_params': 135791104},
 {'encoder_a_params': 24141824,
  'aux_decoder_a_params': 12079104,
  'main_a_params': 30164992,
  'decoder_a_params': 18143232,
  'parallel_a_params': 25174016,
  'ar_decoder_a_params': 74474496,
  'model_a_params': 208319488,
  'vocab_a_params': 71172096,
  'total_a_params': 279491584,
  'infer_model_a_params': 172098560,
  'infer_vocab_a_params': 25755648,
  'infer_total_a_params': 197854208})

In [9]:
model

Model(
  (vocab_emb): Embedding(50304, 512)
  (ar_vocab_emb): Embedding(50304, 512)
  (encoder): Encoder(
    (layers): ModuleList(
      (0-3): 4 x Layer(
        (attn): Linear(in_features=512, out_features=1536, bias=False)
        (attn_o): Linear(in_features=512, out_features=512, bias=False)
        (ffn_up): Linear(in_features=512, out_features=2560, bias=False)
        (ffn_down): Linear(in_features=1280, out_features=512, bias=False)
        (attn_norm): RMSNorm((512,), eps=1e-06, elementwise_affine=True)
        (ffn_norm): RMSNorm((512,), eps=1e-06, elementwise_affine=True)
      )
    )
    (attention): Attention(
      (rotary_emb): RotaryEmb()
    )
  )
  (aux_decoder): AuxDecoder(
    (layers): ModuleList(
      (0-1): 2 x Layer(
        (attn): Linear(in_features=512, out_features=1536, bias=False)
        (attn_o): Linear(in_features=512, out_features=512, bias=False)
        (ffn_up): Linear(in_features=512, out_features=2560, bias=False)
        (ffn_down): Linear(in

In [58]:
import random

In [60]:
eot_idx = 1
pad_idx = 0
b = 8
s = 64
block_size = 8
buffer = torch.randint(1, 16, (512,))

In [1]:
tokens = torch.full((b, s + 1), pad_idx, dtype=torch.long)
eot = torch.argwhere(buffer == eot_idx).squeeze(-1)
eot_num = eot.shape[0]
b_pos_left, b_pos_right, eot_pos = 0, 0, 0

for row in range(b):
    pos = 1
    while pos <= s:
        pad_num = random.randint(0, block_size - 1)
        pos += pad_num
        rest_len = s + 1 - pos

        if eot_pos >= eot_num or (
            eot_pos < eot_num and eot[eot_pos] - b_pos_left + 1 > rest_len
        ):
            b_pos_right = b_pos_left + rest_len
            tokens[row, pos:] = buffer[b_pos_left:b_pos_right]
            pos = s + 1
            b_pos_left = b_pos_right
        else:
            b_pos_right = eot[eot_pos] + 1
            s_len = b_pos_right - b_pos_left
            tokens[row, pos : pos + s_len] = buffer[b_pos_left:b_pos_right]
            end_pad = (block_size - (pos + s_len - 1) % block_size) % block_size
            pos = pos + s_len + end_pad
            b_pos_left = b_pos_right
            eot_pos += 1

NameError: name 'torch' is not defined

In [64]:
tokens[:, 1:].reshape(64, 8)

tensor([[ 5, 10,  3,  4,  7, 15, 11,  1],
        [ 0,  0,  0,  0,  0,  9,  4,  2],
        [ 1,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0, 14,  8,  5,  7, 13],
        [12, 14, 14, 10,  9, 10,  9, 12],
        [ 9,  8,  6, 12, 15,  1,  0,  0],
        [ 0,  0,  0,  0,  0,  5,  5,  2],
        [ 4,  1,  0,  0,  0,  0,  0,  0],
        [ 0,  4,  2,  5,  4, 10, 15, 10],
        [11,  8,  8, 10,  7,  2, 11, 12],
        [ 9, 10,  5, 10,  2,  3, 11, 14],
        [ 3, 11,  2, 10, 12,  7, 14,  1],
        [12,  9,  3,  7,  6,  9, 13,  4],
        [ 3, 14,  3,  5, 10, 11, 15, 13],
        [ 6,  9, 12, 15,  7, 14,  4,  2],
        [10,  9, 14, 14,  5,  9,  7,  1],
        [ 0,  0,  0,  0,  0,  0,  4,  8],
        [ 6,  3, 12,  8,  4,  7, 10, 13],
        [14,  7,  7,  1,  0,  0,  0,  0],
        [ 0,  0,  0,  9,  2, 14, 14,  5],
        [10,  4,  7, 15, 11,  8,  6,  8],
        [ 7,  1,  0,  0,  0,  0,  0,  0],
        [ 0,  0, 12,  7, 10,  5, 13, 13],
        [ 3,  5,  7, 13,  8, 13,  

In [14]:
e = 2
encoder_global_attn = False
doc = torch.tensor([1, 1, 2, 2, 2, 3]).unsqueeze(0)

In [15]:
def mask_mod(b, h, q_idx, kv_idx):
    q_b_idx = q_idx // e
    kv_b_idx = kv_idx // e
    if encoder_global_attn:
        causal_mask = q_b_idx >= kv_b_idx
        doc_mask = doc[b, q_b_idx] == doc[b, kv_b_idx]
        return causal_mask & doc_mask
    else:
        block_mask = q_b_idx == kv_b_idx
        return block_mask

In [16]:
result = torch.full((12, 12), 2)
for i in range(12):
    for j in range(12):
        result[i, j] = mask_mod(0, 0, i, j)

In [17]:
result

tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]])

In [44]:
e = 4
n = 2
encoder_global_attn = False
old = True
new = True
doc = torch.tensor([1, 1, 2]).unsqueeze(0)

In [45]:
def mask_mod(b, h, q_idx, kv_idx):
    q_b_idx = q_idx // e
    kv_b_idx = kv_idx // e
    block_mask = q_b_idx == kv_b_idx
    if encoder_global_attn:
        q_r_idx = q_idx % e
        kv_r_idx = kv_idx % e
        q_old = q_r_idx < n
        q_new = q_r_idx >= n
        kv_old = kv_r_idx < n
        kv_cond = True if new else kv_old
        causal_mask = q_b_idx >= kv_b_idx
        doc_mask = doc[b, q_b_idx] == doc[b, kv_b_idx]
        return (
            q_old & kv_old & causal_mask & doc_mask
            | q_new & kv_cond & block_mask
        )
    else:
        if old and new:
            return block_mask
        if not old and not new:
            kv_old = (kv_idx % e) < n
            return block_mask & kv_old
        if old and not new:
            q_new = (q_idx % e) >= n
            kv_new = (kv_idx % e) >= n
            return block_mask & ~(q_new & kv_new)
        if not old and new:
            q_old = (q_idx % e) < n
            kv_new = (kv_idx % e) >= n
            return block_mask & ~(q_old & kv_new)

In [46]:
result = torch.full((12, 12), 2)
for i in range(12):
    for j in range(12):
        result[i, j] = mask_mod(0, 0, i, j)

In [47]:
result

tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]])

In [84]:
k = 2
n = 2
e = k + n
include_previous_block = True

In [119]:
doc = torch.tensor([1, 1, 2]).unsqueeze(0)

In [120]:
doc

tensor([[1, 1, 2]])

In [121]:
def mask_mod_2(b, h, q_idx, kv_idx):
    q_b_idx = q_idx // e
    kv_b_idx = kv_idx // e
    q_r_idx = q_idx % e
    kv_r_idx = kv_idx % e
    q_token = q_r_idx >= k
    kv_token = kv_r_idx >= k
    q_latent = q_r_idx < k
    kv_latent = kv_r_idx < k
    same_block = q_b_idx == kv_b_idx
    causal_mask = q_idx >= kv_idx
    intra_block_token_mask = q_token & same_block & causal_mask
    inter_block_token_mask = False
    if include_previous_block:
        previous_block = q_b_idx == kv_b_idx + 1
        doc_mask = doc[b, q_b_idx] == doc[b, kv_b_idx]
        inter_block_token_mask = q_token & kv_token & previous_block & doc_mask
    intra_block_latent_mask = q_latent & kv_latent & same_block
    return (
        intra_block_token_mask
        | inter_block_token_mask
        | intra_block_latent_mask
    )

In [122]:
result = torch.full((12, 12), 2)
for i in range(12):
    for j in range(12):
        result[i, j] = mask_mod_2(0, 0, i, j)

In [123]:
result

tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]])

In [60]:
m = 3
n = 3
w = 3
e = m + n
doc = torch.tensor([1, 1]).unsqueeze(0)

In [61]:
def mask_mod(b, h, q_idx, kv_idx):
    q_b_idx = q_idx // e
    kv_b_idx = kv_idx // e
    q_r_idx = q_idx % e
    kv_r_idx = kv_idx % e
    q_new = q_r_idx >= m
    kv_new = kv_r_idx >= m
    q_old = q_r_idx < m
    kv_old = kv_r_idx < m
    same_block = q_b_idx == kv_b_idx
    causal_mask = q_idx >= kv_idx
    kv_b_idx_m = max(kv_b_idx - 1, 0)
    doc_mask = doc[b, q_b_idx] == doc[b, kv_b_idx]
    old_doc_mask = doc[b, q_b_idx] == doc[b, kv_b_idx_m]
    window_mask = q_r_idx + q_b_idx * n < kv_r_idx + kv_b_idx * n + w

    new_new_mask = q_new & kv_new & causal_mask & window_mask & doc_mask
    any_old_mask = (q_new & old_doc_mask | q_old) & kv_old & same_block
    return new_new_mask | any_old_mask

In [62]:
result = torch.full((12, 12), 2)
for i in range(12):
    for j in range(12):
        result[i, j] = mask_mod(0, 0, i, j)

In [63]:
result

tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]])

In [39]:
def mask_mod(b, h, q_idx, kv_idx):
    q_b_idx = q_idx // e
    kv_b_idx = kv_idx // e
    q_r_idx = q_idx % e
    kv_r_idx = kv_idx % e
    q_token = q_r_idx >= k
    kv_token = kv_r_idx >= k
    q_latent = q_r_idx < k
    kv_latent = kv_r_idx < k
    same_block = q_b_idx == kv_b_idx
    causal_mask = q_idx >= kv_idx
    kv_b_idx_m = max(kv_b_idx - 1, 0)
    doc_mask = doc[b, q_b_idx] == doc[b, kv_b_idx]
    latent_doc_mask = doc[b, q_b_idx] == doc[b, kv_b_idx_m]
    window_mask = q_b_idx < kv_b_idx + a

    token_token_mask = q_token & kv_token & causal_mask & window_mask & doc_mask
    token_latent_mask = q_token & kv_latent & same_block & latent_doc_mask
    latent_latent_mask = q_latent & kv_latent & same_block
    return (
        token_token_mask
        | token_latent_mask
        | latent_latent_mask
    )

In [40]:
k = 2
n = 2
e = k + n
a = 2

In [41]:
doc = torch.tensor([1, 1, 2]).unsqueeze(0)

In [42]:
result = torch.full((12, 12), 2)
for i in range(12):
    for j in range(12):
        result[i, j] = mask_mod(0, 0, i, j)

In [43]:
result

tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]])

In [44]:
def mask_mod_3(b, h, q_idx, kv_idx):
    q_b_idx = q_idx // e
    kv_b_idx = kv_idx // e
    q_r_idx = q_idx % e
    kv_r_idx = kv_idx % e
    block_mask = q_b_idx == kv_b_idx
    latent_mask = q_r_idx == kv_r_idx
    causal_mask = q_b_idx >= kv_b_idx
    doc_mask = doc[b, q_b_idx] == doc[b, kv_b_idx]
    return block_mask | (latent_mask & causal_mask & doc_mask)

In [47]:
e = 2

In [48]:
doc = torch.tensor([1, 1, 1, 2, 2, 3]).unsqueeze(0)

In [49]:
result_3 = torch.full((12, 12), 2)
for i in range(12):
    for j in range(12):
        result_3[i, j] = mask_mod_3(0, 0, i, j)

In [50]:
result_3

tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]])